# ABoffsets_DES_DR2_stat_err

Created:  2021.01.04

**Let us consider the "precision" (statistical error) in the absolute calibration to mean how well we have tied the data in the DES footprint to the "Truth" -- whether that Truth is a spectrum of C26202 from the HST CalSpec database or an ensemble of DA white dwarf models of a "Golden sample" of DA WDs scattered over the DES footprint.  In this case, we are most interested in the DES observational errors rather than the errors in the spectrophotometric standards we are tying the DES to.  We cover this case in this Jupyter notebook.**

**Let us thus consider the "accuracy" (systematic error) in the absolute calibration to mean how True is the Truth (how well do the C26202 or the ensemble of DA WD models reflect reality).  We cover this in a separate Jupyter notebook.** 

## 1. Initial Setup...

In [1]:
import numpy as np
import pandas as pd
import glob
import math
import os

import matplotlib.pyplot as plt


%matplotlib inline

## 2. Input Files...

In [2]:
# Input file...
dirName='/data/des40.a/data/dtucker/Y6A1_abscal/SynthMags/CalSpec/v2020-04-27'
inputFile_calspec = dirName+'/y6a2CoaddSummary_match_best_36arcsec.combined.csv'

## 3.  Analysis...

### Read in file...

In [3]:
calspec_df = pd.read_csv(inputFile_calspec)
mask1 = calspec_df.NEPOCHS_G.notnull()
mask2 = calspec_df.Separation < 5.0
mask3 = np.abs(calspec_df.MAG_AUTO_I - calspec_df.i) < 0.5
mask = mask1 & mask2 & mask3
calspec_df = calspec_df[mask].copy().reset_index(drop=True)

### MAG_AUTO Analysis...

In [4]:
calspec_df['gr_std'] = (calspec_df['MAG_AUTO_G'] - calspec_df['MAG_AUTO_R'])
calspec_df['ri_std'] = (calspec_df['MAG_AUTO_R'] - calspec_df['MAG_AUTO_I'])
calspec_df['iz_std'] = (calspec_df['MAG_AUTO_I'] - calspec_df['MAG_AUTO_Z'])
calspec_df['zY_std'] = (calspec_df['MAG_AUTO_Z'] - calspec_df['MAG_AUTO_Y'])
calspec_df['gi_std'] = (calspec_df['MAG_AUTO_G'] - calspec_df['MAG_AUTO_I'])

calspec_df['gr_std_err']=np.sqrt(calspec_df['MAGERR_AUTO_G']*calspec_df['MAGERR_AUTO_G'] + \
                                 calspec_df['MAGERR_AUTO_R']*calspec_df['MAGERR_AUTO_R'] )
calspec_df['ri_std_err']=np.sqrt(calspec_df['MAGERR_AUTO_R']*calspec_df['MAGERR_AUTO_R'] + \
                                 calspec_df['MAGERR_AUTO_I']*calspec_df['MAGERR_AUTO_I'] )
calspec_df['iz_std_err']=np.sqrt(calspec_df['MAGERR_AUTO_I']*calspec_df['MAGERR_AUTO_I'] + \
                                 calspec_df['MAGERR_AUTO_Z']*calspec_df['MAGERR_AUTO_Z'] )
calspec_df['zY_std_err']=np.sqrt(calspec_df['MAGERR_AUTO_Z']*calspec_df['MAGERR_AUTO_Z'] + \
                                 calspec_df['MAGERR_AUTO_Y']*calspec_df['MAGERR_AUTO_Y'] )
calspec_df['gi_std_err']=np.sqrt(calspec_df['MAGERR_AUTO_G']*calspec_df['MAGERR_AUTO_G'] + \
                                 calspec_df['MAGERR_AUTO_I']*calspec_df['MAGERR_AUTO_I'] )

calspec_df['gi'] = ( calspec_df['gr'] + calspec_df['ri'] )

calspec_df['avgerr_gr'] = 0.0
calspec_df['avgerr_ri'] = 0.0
calspec_df['avgerr_iz'] = 0.0
calspec_df['avgerr_zY'] = 0.0
calspec_df['avgerr_gi'] = 0.0

calspec_df['delta_STD_gr'] = calspec_df['gr_std'] - calspec_df['gr']
calspec_df['delta_STD_ri'] = calspec_df['ri_std'] - calspec_df['ri']
calspec_df['delta_STD_iz'] = calspec_df['iz_std'] - calspec_df['iz']
calspec_df['delta_STD_zY'] = calspec_df['zY_std'] - calspec_df['zY']

calspec_df['Err_STD_gr'] = np.sqrt(calspec_df['MAGERR_AUTO_G']*calspec_df['MAGERR_AUTO_G'] + \
                                   calspec_df['MAGERR_AUTO_R']*calspec_df['MAGERR_AUTO_R'] + \
                                   calspec_df['avgerr_gr']*calspec_df['avgerr_gr'])

calspec_df['Err_STD_ri'] = np.sqrt(calspec_df['MAGERR_AUTO_R']*calspec_df['MAGERR_AUTO_R'] + \
                                   calspec_df['MAGERR_AUTO_I']*calspec_df['MAGERR_AUTO_I'] + \
                                   calspec_df['avgerr_ri']*calspec_df['avgerr_ri'])

calspec_df['Err_STD_iz'] = np.sqrt(calspec_df['MAGERR_AUTO_I']*calspec_df['MAGERR_AUTO_I'] + \
                                   calspec_df['MAGERR_AUTO_Z']*calspec_df['MAGERR_AUTO_Z'] + \
                                   calspec_df['avgerr_iz']*calspec_df['avgerr_iz'])

calspec_df['Err_STD_zY'] = np.sqrt(calspec_df['MAGERR_AUTO_Z']*calspec_df['MAGERR_AUTO_Z'] + \
                                   calspec_df['MAGERR_AUTO_Y']*calspec_df['MAGERR_AUTO_Y'] + \
                                   calspec_df['avgerr_zY']*calspec_df['avgerr_zY'])

#print  calspec_df['MAGERR_AUTO_I'], calspec_df['WAVG_MAGERR_PSF_I']

#calspec_df
#calspec_df[['objShortName','gr']]
#calspec_df[['objShortName','MAGERR_AUTO_I','Err_STD_gr','Err_STD_ri','Err_STD_iz','Err_STD_zY']]
calspec_df[['objShortName','MAGERR_AUTO_G','MAGERR_AUTO_R','MAGERR_AUTO_I','MAGERR_AUTO_Z','MAGERR_AUTO_Y']]

,objShortName,MAGERR_AUTO_G,MAGERR_AUTO_R,MAGERR_AUTO_I,MAGERR_AUTO_Z,MAGERR_AUTO_Y
0,c26202,0.000368,0.000319,0.000416,0.000584,0.002509
1,c26202,0.000368,0.000319,0.000416,0.000584,0.002509
2,wd0320_539,0.000202,0.000351,0.000310,0.000588,0.002028
3,wd0320_539,0.000202,0.000351,0.000310,0.000588,0.002028


In [5]:
# Let us consider the "precision" (statisical error) in the absolute calibration 
# to mean how well we have tied the data in the DES footprint to the "Truth" -- 
# whether that Truth is a spectrum of C26202 from the HST CalSpec database or an 
# ensemble of DA white dwarf models of a "Golden sample" of DA WDs scattered over 
# the DES footprint.  We cover this in this Jupyter notebook.
#
# We will consider "accuracy" (systematic error) in the absolute calibration
# to mean how True is the Truth (how well do the C26202 or the ensemble of 
# DA WD models reflect reality).  We cover this in a separate Jupyter notebook. 

In [6]:
# Table 1 Absolute Calibration Precision 
#  (based on observations of C26202, which
#   are the absolute calibration "as provided"):
#
# For individual mags, assume statistical error is Poisson flux 
#  error and coadd uniformity error added in quadrature:
print 'g_rms_stat:', math.sqrt(0.00037*0.00037 + 0.00215*0.00215)
print 'r_rms_stat:', math.sqrt(0.00032*0.00032 + 0.00215*0.00215)
print 'i_rms_stat:', math.sqrt(0.00042*0.00042 + 0.00215*0.00215)
print 'z_rms_stat:', math.sqrt(0.00058*0.00058 + 0.00215*0.00215)
print 'Y_rms_stat:', math.sqrt(0.00251*0.00251 + 0.00215*0.00215)

# Note: since we are concerned how well the data in the whole
#  DES footprint is tied to C26202, we include the uniformity
#  error (e.g., what if C26202 were located in a different part
#  of the footprint -- in that case, its observed mags would likely
#  be different, since the DES is not perfectly uniform photometrically.)

g_rms_stat: 0.00218160491382
r_rms_stat: 0.00217368350962
i_rms_stat: 0.00219063917613
z_rms_stat: 0.00222685877415
Y_rms_stat: 0.00330493570285


In [7]:
# Table 2 Absolute Calibration Precision for C26202:

# For colors, assume statistical error is Poisson flux 
#  errors of the two mags added in quadrature (but that 
#  coadd uniformity error cancels out -- i.e.. that the
#  offsets in uniformity are 100% correlated across the
#  DES passbands):
print 'i_rms_stat:', math.sqrt(0.00042*0.00042 + 0.00215*0.00215)
print '(g-r)_rms_stat:', math.sqrt(0.00037*0.00037 + 0.00032*0.00032)
print '(r-i)_rms_stat:', math.sqrt(0.00032*0.00032 + 0.00042*0.00042)
print '(i-z)_rms_stat:', math.sqrt(0.00042*0.00042 + 0.00058*0.00058)
print '(z-Y)_rms_stat:', math.sqrt(0.00058*0.00058 + 0.00251*0.00251)

i_rms_stat: 0.00219063917613
(g-r)_rms_stat: 0.000489182992345
(r-i)_rms_stat: 0.000528015151298
(i-z)_rms_stat: 0.000716100551599
(z-Y)_rms_stat: 0.00257614052412


### WAVG_MAG_PSF Analysis...

In [8]:
calspec_df['gr_std'] = (calspec_df['WAVG_MAG_PSF_G'] - calspec_df['WAVG_MAG_PSF_R'])
calspec_df['ri_std'] = (calspec_df['WAVG_MAG_PSF_R'] - calspec_df['WAVG_MAG_PSF_I'])
calspec_df['iz_std'] = (calspec_df['WAVG_MAG_PSF_I'] - calspec_df['WAVG_MAG_PSF_Z'])
calspec_df['zY_std'] = (calspec_df['WAVG_MAG_PSF_Z'] - calspec_df['WAVG_MAG_PSF_Y'])
calspec_df['gi_std'] = (calspec_df['WAVG_MAG_PSF_G'] - calspec_df['WAVG_MAG_PSF_I'])

calspec_df['gr_std_err']=np.sqrt(calspec_df['WAVG_MAGERR_PSF_G']*calspec_df['WAVG_MAGERR_PSF_G'] + \
                                 calspec_df['WAVG_MAGERR_PSF_R']*calspec_df['WAVG_MAGERR_PSF_R'] )
calspec_df['ri_std_err']=np.sqrt(calspec_df['WAVG_MAGERR_PSF_R']*calspec_df['WAVG_MAGERR_PSF_R'] + \
                                 calspec_df['WAVG_MAGERR_PSF_I']*calspec_df['WAVG_MAGERR_PSF_I'] )
calspec_df['iz_std_err']=np.sqrt(calspec_df['WAVG_MAGERR_PSF_I']*calspec_df['WAVG_MAGERR_PSF_I'] + \
                                 calspec_df['WAVG_MAGERR_PSF_Z']*calspec_df['WAVG_MAGERR_PSF_Z'] )
calspec_df['zY_std_err']=np.sqrt(calspec_df['WAVG_MAGERR_PSF_Z']*calspec_df['WAVG_MAGERR_PSF_Z'] + \
                                 calspec_df['WAVG_MAGERR_PSF_Y']*calspec_df['WAVG_MAGERR_PSF_Y'] )
calspec_df['gi_std_err']=np.sqrt(calspec_df['WAVG_MAGERR_PSF_G']*calspec_df['WAVG_MAGERR_PSF_G'] + \
                                 calspec_df['WAVG_MAGERR_PSF_I']*calspec_df['WAVG_MAGERR_PSF_I'] )

calspec_df['gi'] = ( calspec_df['gr'] + calspec_df['ri'] )

calspec_df['avgerr_gr'] = 0.0
calspec_df['avgerr_ri'] = 0.0
calspec_df['avgerr_iz'] = 0.0
calspec_df['avgerr_zY'] = 0.0
calspec_df['avgerr_gi'] = 0.0

calspec_df['delta_STD_gr'] = calspec_df['gr_std'] - calspec_df['gr']
calspec_df['delta_STD_ri'] = calspec_df['ri_std'] - calspec_df['ri']
calspec_df['delta_STD_iz'] = calspec_df['iz_std'] - calspec_df['iz']
calspec_df['delta_STD_zY'] = calspec_df['zY_std'] - calspec_df['zY']

calspec_df['Err_STD_gr'] = np.sqrt(calspec_df['WAVG_MAGERR_PSF_G']*calspec_df['WAVG_MAGERR_PSF_G'] + \
                                   calspec_df['WAVG_MAGERR_PSF_R']*calspec_df['WAVG_MAGERR_PSF_R'] + \
                                   calspec_df['avgerr_gr']*calspec_df['avgerr_gr'])

calspec_df['Err_STD_ri'] = np.sqrt(calspec_df['WAVG_MAGERR_PSF_R']*calspec_df['WAVG_MAGERR_PSF_R'] + \
                                   calspec_df['WAVG_MAGERR_PSF_I']*calspec_df['WAVG_MAGERR_PSF_I'] + \
                                   calspec_df['avgerr_ri']*calspec_df['avgerr_ri'])

calspec_df['Err_STD_iz'] = np.sqrt(calspec_df['WAVG_MAGERR_PSF_I']*calspec_df['WAVG_MAGERR_PSF_I'] + \
                                   calspec_df['WAVG_MAGERR_PSF_Z']*calspec_df['WAVG_MAGERR_PSF_Z'] + \
                                   calspec_df['avgerr_iz']*calspec_df['avgerr_iz'])

calspec_df['Err_STD_zY'] = np.sqrt(calspec_df['WAVG_MAGERR_PSF_Z']*calspec_df['WAVG_MAGERR_PSF_Z'] + \
                                   calspec_df['WAVG_MAGERR_PSF_Y']*calspec_df['WAVG_MAGERR_PSF_Y'] + \
                                   calspec_df['avgerr_zY']*calspec_df['avgerr_zY'])

#print  calspec_df['WAVG_MAGERR_PSF_I'], calspec_df['WAVG_MAGERR_PSF_I']

#calspec_df
#calspec_df[['objShortName','gr']]
calspec_df[['objShortName','WAVG_MAGERR_PSF_I','Err_STD_gr','Err_STD_ri','Err_STD_iz','Err_STD_zY','NEPOCHS_G','NEPOCHS_R','NEPOCHS_I','NEPOCHS_Z','NEPOCHS_Y']]
calspec_df[['objShortName','WAVG_MAGERR_PSF_G','WAVG_MAGERR_PSF_R','WAVG_MAGERR_PSF_I','WAVG_MAGERR_PSF_Z','WAVG_MAGERR_PSF_Y','NEPOCHS_G','NEPOCHS_R','NEPOCHS_I','NEPOCHS_Z','NEPOCHS_Y']]

,objShortName,WAVG_MAGERR_PSF_G,WAVG_MAGERR_PSF_R,WAVG_MAGERR_PSF_I,WAVG_MAGERR_PSF_Z,WAVG_MAGERR_PSF_Y,NEPOCHS_G,NEPOCHS_R,NEPOCHS_I,NEPOCHS_Z,NEPOCHS_Y
0,c26202,0.000416,0.000348,0.000344,0.000434,0.001347,9.0,8.0,9.0,9.0,6.0
1,c26202,0.000416,0.000348,0.000344,0.000434,0.001347,9.0,8.0,9.0,9.0,6.0
2,wd0320_539,0.000299,0.000633,0.000304,0.000410,0.001179,3.0,1.0,6.0,7.0,6.0
3,wd0320_539,0.000299,0.000633,0.000304,0.000410,0.001179,3.0,1.0,6.0,7.0,6.0


In [9]:
# Table 1 Absolute Calibration Precision 
#  (based on observations of C26202, which
#   are the absolute calibration "as provided"):
#
# For individual mags, assume statistical error is Poisson flux 
#  error and coadd uniformity error added in quadrature:
print 'g_rms_stat:', math.sqrt(0.00042*0.00042 + 0.00215*0.00215)
print 'r_rms_stat:', math.sqrt(0.00035*0.00035 + 0.00215*0.00215)
print 'i_rms_stat:', math.sqrt(0.00034*0.00034 + 0.00215*0.00215)
print 'z_rms_stat:', math.sqrt(0.00043*0.00043 + 0.00215*0.00215)
print 'Y_rms_stat:', math.sqrt(0.00135*0.00135 + 0.00215*0.00215)

# Note: since we are concerned how well the data in the whole
#  DES footprint is tied to C26202, we include the uniformity
#  error (e.g., what if C26202 were located in a different part
#  of the footprint -- in that case, its observed mags would likely
#  be different, since the DES is not perfectly uniform photometrically.)

g_rms_stat: 0.00219063917613
r_rms_stat: 0.00217830209108
i_rms_stat: 0.00217671771252
z_rms_stat: 0.00219257839084
Y_rms_stat: 0.00253870045496


In [10]:
# Table 2 Absolute Calibration Precision for C26202:

# For colors, assume statistical error is Poisson flux 
#  errors of the two mags added in quadrature (but that 
#  coadd uniformity error cancels out -- i.e.. that the
#  offsets in uniformity are 100% correlated across the
#  DES passbands):
print 'i_rms_stat:', math.sqrt(0.00034*0.00034 + 0.00215*0.00215)
print '(g-r)_rms_stat:', math.sqrt(0.00042*0.00042 + 0.00035*0.00035)
print '(r-i)_rms_stat:', math.sqrt(0.00035*0.00035 + 0.00034*0.00034)
print '(i-z)_rms_stat:', math.sqrt(0.00034*0.00034 + 0.00043*0.00043)
print '(z-Y)_rms_stat:', math.sqrt(0.00043*0.00043 + 0.00135*0.00135)

i_rms_stat: 0.00217671771252
(g-r)_rms_stat: 0.000546717477313
(r-i)_rms_stat: 0.00048795491595
(i-z)_rms_stat: 0.000548178802947
(z-Y)_rms_stat: 0.00141682744186
